In [ ]:
import copy
import os
import sys
import torch
import numpy as np

from ... import Agent

In [ ]:
# define the testing environment
env_templates = []

# define the target locs to place the objects
target_locs = []

In [ ]:
# load the models
model = torch.load('')


In [ ]:
# load cfg
cfg = 

In [ ]:
# generate the maps 
num_reps = 100
ixs = 0

# parameters
world_size = env_templates[0].height
num_reps = 100

v_heatmap_lst = []

# main
for env in env_templates: 

    # find the empty locations
    empty_locs = []
    for i in range(env.world.shape[0]):
        for j in range(env.world.shape[1]):
            if str(env.world[i,j,0]) == 'EmptyObject':  
                empty_locs.append((i,j,0)) 
    
    # create an empty template for the heatmap 
    v_heatmap = np.empty((world_size,world_size))
    v_heatmap[:] = np.nan

    # iteratively place the agent on every empty location in the env, 
    # then record the model's output (Q/V/...), and store it in the 
    # heatmap template 

    if target_locs is not None:
        available_locs = target_locs
    else:
        available_locs = empty_locs

    for loc in available_locs: 
        env_ = copy.deepcopy(env)
        agent = Agent(model=model, cfg=cfg, ixs=ixs)
        # place the agent
        env_.world[loc] = agent
        agent.location = loc
        # generate the agent's state
        state = agent.pov(env_)

        val_estimation_trials = [] 

        # this loop is tailored for the model used in my experiment 
        for _ in range(num_reps):

            act, vs, Qs = model.take_action(state, eval=True)
            max_q = float(torch.max(Qs))
            v = float(torch.mean(vs, dim=0))     
            val_estimation_trials.append(max_q) 
        
        # store the value of interest in the heatmap 
        v_m = np.mean(val_estimation_trials)
        v_heatmap[loc[0], loc[1]] = v_m 

    v_heatmap_lst.append(v_heatmap)

